# Learning how to build deep neural nets
## Based on Chap 10 of Hands on ML 3rd ed

In [1]:
import sys
import sklearn
import tensorflow as tf
from tensorflow import keras
import numpy as np
import os
np.random.seed(0)
import matplotlib as mpl
import matplotlib.pyplot as plt

C:\anaconda_python\envs\insight_2\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
C:\anaconda_python\envs\insight_2\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
C:\anaconda_python\envs\insight_2\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
C:\anaconda_python\envs\insight_2\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
C:\anaconda_python\envs\insight_2\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may

In [2]:
# There's some numpy incompatibility with tf which will probably get fixed soon. Downgrading to 1.16.1 is a temporary fix
np.__version__

'1.18.1'

### List of initializer functions, glorot_uniform is usually the default.
### He initializer is also one that works well with Relu

In [3]:
[name for name in dir(keras.initializers) if not name.startswith("_")]

['Constant',
 'GlorotNormal',
 'GlorotUniform',
 'Identity',
 'Initializer',
 'Ones',
 'Orthogonal',
 'RandomNormal',
 'RandomUniform',
 'TruncatedNormal',
 'VarianceScaling',
 'Zeros',
 'constant',
 'deserialize',
 'get',
 'glorot_normal',
 'glorot_uniform',
 'he_normal',
 'he_uniform',
 'identity',
 'lecun_normal',
 'lecun_uniform',
 'ones',
 'orthogonal',
 'serialize',
 'zeros']

In [5]:
keras.layers.Dense(10, activation='relu', kernel_initializer='he_normal')

In [6]:
# To get He initializer with uniform distribution but based on fan_avg rather than fan_in
init = keras.initializers.VarianceScaling(scale=2., mode='fan_avg', distribution='uniform')
keras.layers.Dense(10, activation="relu", kernel_initializer=init)

### List of activation functions

In [7]:
[m for m in dir(keras.activations) if not m.startswith("_")]

['deserialize',
 'elu',
 'exponential',
 'get',
 'hard_sigmoid',
 'linear',
 'relu',
 'selu',
 'serialize',
 'sigmoid',
 'softmax',
 'softplus',
 'softsign',
 'tanh']

1. Sigmoid function is in biological neurons but lead to exploding and vanishing gradients so sometimes ReLU is better.
2. Drawback of ReLU - dying neurons, stop outputting anything other than zero
3. To solve this - leaky ReLU = max(\alpha z , z) alpha usually set to 0.01
4. Newer paper used something called exponential linear unit (ELU) - slower to compute
5. 2017 - Scaled ELU (SELU). Exclusively a stack of dense layers which self normalize. Initialization with lecun_normal. Architecture must be sequential, can improve CNN's as well

In [8]:
# Can't set as activation directly 
[m for m in dir(keras.layers) if "relu" in m.lower()]

['LeakyReLU', 'PReLU', 'ReLU', 'ThresholdedReLU']

### Try to train fashion MNIST using Leaky ReLU

In [10]:
(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.fashion_mnist.load_data()
X_train_full = X_train_full / 255.0
X_test = X_test / 255.0
X_valid, X_train = X_train_full[:5000], X_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

In [11]:
tf.random.set_seed(0)
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(300, kernel_initializer="he_normal"),
    # Have to use this additional layer after each layer
    keras.layers.LeakyReLU(),
    keras.layers.Dense(100, kernel_initializer="he_normal"),
    keras.layers.LeakyReLU(),
    keras.layers.Dense(10, activation="softmax")
])

In [12]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=keras.optimizers.SGD(lr=1e-3),
              metrics=["accuracy"])


In [13]:
history = model.fit(X_train, y_train, epochs=10,
                    validation_data=(X_valid, y_valid))

Train on 55000 samples, validate on 5000 samples
Epoch 1/10
55000/55000 [==============================] - 5s 100us/sample - loss: 1.2530 - accuracy: 0.6110 - val_loss: 0.8560 - val_accuracy: 0.7272
Epoch 2/10
55000/55000 [==============================] - 5s 86us/sample - loss: 0.7813 - accuracy: 0.7411 - val_loss: 0.6964 - val_accuracy: 0.7694
Epoch 3/10
55000/55000 [==============================] - 5s 86us/sample - loss: 0.6735 - accuracy: 0.7742 - val_loss: 0.6251 - val_accuracy: 0.7982
Epoch 4/10
55000/55000 [==============================] - 5s 84us/sample - loss: 0.6162 - accuracy: 0.7950 - val_loss: 0.5851 - val_accuracy: 0.8056
Epoch 5/10
55000/55000 [==============================] - 5s 83us/sample - loss: 0.5788 - accuracy: 0.8086 - val_loss: 0.5528 - val_accuracy: 0.8180
Epoch 6/10
55000/55000 [==============================] - 5s 82us/sample - loss: 0.5523 - accuracy: 0.8160 - val_loss: 0.5279 - val_accuracy: 0.8272
Epoch 7/10
55000/55000 [==============================] 

### Training using SELU
In this case need to scale to mean 0 and SD 1

In [14]:
pixel_means = X_train.mean(axis=0, keepdims=True)
pixel_stds = X_train.std(axis=0, keepdims=True)
X_train_scaled = (X_train - pixel_means) / pixel_stds
X_valid_scaled = (X_valid - pixel_means) / pixel_stds
X_test_scaled = (X_test - pixel_means) / pixel_stds

In [15]:
model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[28, 28]))
model.add(keras.layers.Dense(300, activation="selu",
                             kernel_initializer="lecun_normal"))
for layer in range(99):
    model.add(keras.layers.Dense(100, activation="selu",
                                 kernel_initializer="lecun_normal"))
model.add(keras.layers.Dense(10, activation="softmax"))

In [16]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=keras.optimizers.SGD(lr=1e-3),
              metrics=["accuracy"])

In [17]:
# Much slower but avoids vanishing or exploding gradient problem
history = model.fit(X_train_scaled, y_train, epochs=5,
                    validation_data=(X_valid_scaled, y_valid))

Train on 55000 samples, validate on 5000 samples
Epoch 1/5
55000/55000 [==============================] - 53s 961us/sample - loss: 1.2577 - accuracy: 0.5144 - val_loss: 0.9184 - val_accuracy: 0.6616
Epoch 2/5
55000/55000 [==============================] - 45s 825us/sample - loss: 0.8798 - accuracy: 0.6705 - val_loss: 1.6595 - val_accuracy: 0.2790
Epoch 3/5
55000/55000 [==============================] - 45s 823us/sample - loss: 0.9829 - accuracy: 0.6283 - val_loss: 0.7660 - val_accuracy: 0.7002
Epoch 4/5
55000/55000 [==============================] - 46s 836us/sample - loss: 0.7512 - accuracy: 0.7079 - val_loss: 0.6577 - val_accuracy: 0.7442
Epoch 5/5
55000/55000 [==============================] - 45s 816us/sample - loss: 0.6259 - accuracy: 0.7650 - val_loss: 0.5664 - val_accuracy: 0.7994
